**Experiment with Descartes Labs API and inventory datasets**

Let's see what datasets are available via the Descartes Labs API.


In [ ]:
# Import plotting packages
%matplotlib inline
import matplotlib.pyplot as plt
import shapely.geometry
import cartopy

# Import Descartes Labs library 
import descarteslabs as dl
from pprint import pprint

In [ ]:
sources = dl.metadata.sources()
pprint (sources)

In [ ]:
# Find potential matches
matches = dl.places.find('honduras')
pprint(matches)
# The first one looks good to me, so lets make that our area of interest.
aoi = matches[0]

In [ ]:
aoi = matches[0]
shape = dl.places.shape(aoi['slug'], geom='low')
bbox = shape['bbox']

# Lets load up the Albers Equal Area projection.
lonlat_crs = cartopy.crs.PlateCarree()
albers = cartopy.crs.AlbersEqualArea(central_latitude=36.0, central_longitude=-105)

fig = plt.figure(figsize=(4, 8))
ax = plt.subplot(projection=albers) # Specify projection of the map here
shp = shapely.geometry.shape(shape['geometry'])

# When adding a geometry in latlon coordinates, specify the latlon projection
ax.add_geometries([shp], lonlat_crs)

# You can set extents in latlon, as long as you specify the projection with `crs`
ax.set_extent((bbox[0], bbox[2], bbox[1], bbox[3]), crs=lonlat_crs)
ax.gridlines(crs=lonlat_crs)
plt.show()

In [ ]:
import json
feature_collection = dl.metadata.search(products='landsat:LC08:PRE:TOAR', start_time='2017-03-12',
                                        end_time='2015-03-20', limit=10, place=aoi['slug'])

print len(feature_collection['features'])

In [ ]:
arr, meta = dl.raster.ndarray(
    f0['id'],
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Byte',
    resolution=30,
    cutline=shape['geometry'],
)


In [ ]:
plt.figure(figsize=[16,16])
plt.imshow(arr)

**Test technique for classification of Landsat imagery**

Using scikit-learn learn and gdal, we can create training data to classify urban areas in imagery. The t


In [17]:
import numpy as np
import os

from osgeo import gdal
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier